In [1]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy

# spacy is used to work on text
import spacy

#import other libraries
import numpy as np
import pandas as pd
import random
import sys
import os
import time
import codecs
import collections
import string
import re
from six.moves import cPickle

Using TensorFlow backend.


In [2]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')

In [25]:
df.Plot[0]#.split(' ')

"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"

In [3]:
def create_wordlist(doc):
    wl = []
    for word in re.split(' |\r\n',doc):
        word = word.lower()
        punc = string.punctuation.replace("'", "")
        if '[' in word and ']' in word:
            start = word.index('[')
            end = word.index(']')
            word = word[:start] + word[end+1:]
        for i in punc:
            word = word.replace(i, "")
        wl.append(word.lower())
    return wl

In [17]:
df['Plot'].shape

(34886,)

In [5]:
len(df.Plot[1].split(' '))

86

In [6]:
start = time.time()
wordlist = []

for i in df['Plot'][:100]:
    wl = create_wordlist(i)
    wordlist = wordlist + wl
print("Wall time: {} seconds".format(time.time() - start))

Wall time: 0.08117413520812988 seconds


In [7]:
len(wordlist)

15339

In [110]:
len(wordlist)

12993831

In [8]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

# #save the words and vocabulary
# with open('word_dict.txt', 'wb') as f:
#     cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  3600


In [15]:
rnn_size = 256 # size of RNN
seq_length = 10 # sequence length
learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [7]:
# with open('word_dict.txt', 'rb') as f:
#     w = cPickle.load(f)

In [ ]:
word

In [16]:
#create sequences
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 15329


In [14]:
15259 * 10 * 3600 / 1024.0**3

0.5115978419780731

In [17]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

In [18]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [19]:
rnn_size = 256 # size of RNN
seq_length = 10 # sequence length
learning_rate = 0.001 #learning rate

md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


model built!
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               7899136   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3600)              1846800   
_________________________________________________________________
activation_1 (Activation)    (None, 3600)              0         
Total params: 9,745,936
Trainable params: 9,745,936
Non-trainable params: 0
_________________________________________________________________


In [21]:
batch_size = 32 # minibatch size
num_epochs = 50 # number of epochs

# callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
#            ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
#                            monitor='val_loss', verbose=0, mode='auto', period=2)]
#fit the model
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
#                  callbacks=callbacks,
                 validation_split=0.1)

Train on 13796 samples, validate on 1533 samples
Epoch 1/50
13796/13796 [==============================] - 228s 17ms/step - loss: 6.7177 - categorical_accuracy: 0.0840 - val_loss: 6.4987 - val_categorical_accuracy: 0.0855
Epoch 2/50
13796/13796 [==============================] - 226s 16ms/step - loss: 6.2706 - categorical_accuracy: 0.0850 - val_loss: 6.5930 - val_categorical_accuracy: 0.0855
Epoch 3/50
13796/13796 [==============================] - 226s 16ms/step - loss: 6.0963 - categorical_accuracy: 0.0853 - val_loss: 6.6868 - val_categorical_accuracy: 0.0861
Epoch 4/50
13796/13796 [==============================] - 223s 16ms/step - loss: 5.9204 - categorical_accuracy: 0.0857 - val_loss: 6.8409 - val_categorical_accuracy: 0.0887
Epoch 5/50
13796/13796 [==============================] - 225s 16ms/step - loss: 5.7450 - categorical_accuracy: 0.0898 - val_loss: 6.9979 - val_categorical_accuracy: 0.0985
Epoch 6/50
13796/13796 [==============================] - 234s 17ms/step - loss: 5.554

Epoch 48/50
13796/13796 [==============================] - 238s 17ms/step - loss: 0.1025 - categorical_accuracy: 0.9749 - val_loss: 13.7480 - val_categorical_accuracy: 0.0515
Epoch 49/50
13796/13796 [==============================] - 237s 17ms/step - loss: 0.1155 - categorical_accuracy: 0.9728 - val_loss: 13.4590 - val_categorical_accuracy: 0.0417
Epoch 50/50
13796/13796 [==============================] - 241s 17ms/step - loss: 0.1269 - categorical_accuracy: 0.9698 - val_loss: 13.3474 - val_categorical_accuracy: 0.0607


NameError: name 'save_dir' is not defined

In [22]:
#save the model
md.save('my_model_generate_sentences.h5')

In [23]:
#load vocabulary
print("loading vocabulary...")
# vocab_file = os.path.join(save_dir, "words_vocab.pkl")

 
with open('word_dict_short.txt', 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

with open('word_dict_short.txt', 'rb') as f:
    words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

from keras.models import load_model
# load the model
print("loading model...")
model = load_model('my_model_generate_sentences.h5')

loading vocabulary...
loading model...


In [24]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [45]:
vocab

{'': 0,
 "'dick'": 1,
 "'girl'": 2,
 '1422–1461': 3,
 '1455–1487': 4,
 '1470–1471': 5,
 '15': 6,
 '1900': 7,
 '1909': 8,
 '1910': 9,
 '1911': 10,
 '1913': 11,
 '1914': 12,
 '1915': 13,
 '20000': 14,
 '3': 15,
 '3–5': 16,
 '4': 17,
 "45's": 18,
 '4–6': 19,
 '5': 20,
 '6': 21,
 '61': 22,
 '7': 23,
 '8': 24,
 'a': 25,
 'abandon': 26,
 'abandoning': 27,
 'abandons': 28,
 'abduct': 29,
 'abducted': 30,
 'abduction': 31,
 'able': 32,
 'aboard': 33,
 'about': 34,
 'absconded': 35,
 'absence': 36,
 'absent': 37,
 'abuse': 38,
 'abusive': 39,
 'accept': 40,
 'accepts': 41,
 'accident': 42,
 'accidentally': 43,
 'accompanied': 44,
 'accompanying': 45,
 'accosts': 46,
 'accurate': 47,
 'accused': 48,
 'accuses': 49,
 'acknowledged': 50,
 'acquaintance': 51,
 'acquainted': 52,
 'across': 53,
 'act': 54,
 'action': 55,
 'actions': 56,
 'actor': 57,
 'actors': 58,
 "actors'": 59,
 "actress'": 60,
 'ad': 61,
 'adair': 62,
 'adaptation': 63,
 'addicted': 64,
 'adieu—after': 65,
 'adjacent': 66,
 'admi

In [58]:
df.Plot[1]

"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."

In [59]:
words_number = 10 # number of words to generate
seed_sentences = "he is walking past " #seed sentence to start the generating.

#initiate sentences
generated = ''
sentence = []

#we shate the seed accordingly to the neural netwrok needs:
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.33)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

/home/hsiaoyuchiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


a a a a a a he is walking past a thief falls decide to boys where the girl the
